<a href="https://colab.research.google.com/github/Keerthana-956/keerthana/blob/main/SITHAFAL_TASKS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TASK-1:CHAT WITH PDF USING RAG PIPELINE**

In [8]:
!pip install streamlit
import streamlit as st
!pip install PyPDF2
!pip install langchain_community
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.embeddings.spacy_embeddings import SpacyEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.tools.retriever import create_retriever_tool
from dotenv import load_dotenv
!pip install langchain_anthropic
from langchain_anthropic import ChatAnthropic
from langchain.agents import AgentExecutor, create_tool_calling_agent

load_dotenv()
embeddings = SpacyEmbeddings(model_name="en_core_web_sm")
def pdf_read(pdf_doc):
    text = ""
    for pdf in pdf_doc:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text



def get_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_text(text)
    return chunks


def vector_store(text_chunks):

    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_db")


def get_conversational_chain(tools,ques):
    #os.environ["ANTHROPIC_API_KEY"]=os.getenv["ANTHROPIC_API_KEY"]
    llm = ChatAnthropic(model="claude-3-sonnet-20240229", temperature=0, api_key=os.getenv("ANTHROPIC_API_KEY"),verbose=True)

    prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a helpful assistant. Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer""",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)
    tool=[tools]
    agent = create_tool_calling_agent(llm, tool, prompt)

    agent_executor = AgentExecutor(agent=agent, tools=tool, verbose=True)
    response=agent_executor.invoke({"input": ques})
    print(response)
    st.write("Reply: ", response['output'])



def user_input(user_question):



    new_db = FAISS.load_local("faiss_db", embeddings,allow_dangerous_deserialization=True)

    retriever=new_db.as_retriever()
    retrieval_chain= create_retriever_tool(retriever,"pdf_extractor","This tool is to give answer to queries from the pdf")
    get_conversational_chain(retrieval_chain,user_question)





def main():
    st.set_page_config("Chat PDF")
    st.header("RAG based Chat with PDF")

    user_question = st.text_input("Ask a Question from the PDF Files")

    if user_question:
        user_input(user_question)

    with st.sidebar:
        st.title("Menu:")
        pdf_doc = st.file_uploader("Upload your PDF Files and Click on the Submit & Process Button", accept_multiple_files=True)
        if st.button("Submit & Process"):
            with st.spinner("Processing..."):
                raw_text = pdf_read(pdf_doc)
                text_chunks = get_chunks(raw_text)
                vector_store(text_chunks)
                st.success("Done")

if __name__ == "__main__":
    main()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 6.3 MB/s eta 0:00:00


2024-12-21 16:31:44.932 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-21 16:31:44.936 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-21 16:31:45.014 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-12-21 16:31:45.016 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-21 16:31:45.019 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-21 16:31:45.021 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-21 16:31:45.023 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

**TASK2:CHAT WITH WEBSITE USING RAG PIPELINE**

In [10]:
import os

import streamlit as st

from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
!pip install langchain_openai
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter


def set_openai_key(api_key):
    """Set the OpenAI API key in the environment variable.
    Args:
    key (str): The OpenAI API key to set.
    """
    os.environ["OPENAI_API_KEY"] = f"{api_key}"


def get_vectorstore_from_url(url):
    """Retrieve vector store from the given URL.

    This function loads a document from the provided URL,
    splits it into chunks & creates a vector store.

    Args:
        url (str): The URL of the document.

    Returns:
        Chroma: The vector store created from the document chunks.
    """
    # Get text-in documents
    loader = WebBaseLoader(url)
    document = loader.load()

    # Split the documents into chunks
    text_splitter = RecursiveCharacterTextSplitter()
    document_chunks = text_splitter.split_documents(document)

    # Create Vector-store from the chunks:
    embeddings = OpenAIEmbeddings()
    vector_store = Chroma.from_documents(document_chunks, embeddings)
    return vector_store


def get_context_retriever_chain(vector_store):
    """Create a context retriever chain using the given vector store.

    This function creates a history-aware retriever chain
    using a vector store and a chat prompt template.

    Args:
        vector_store (Chroma): The vector store to use.

    Returns:
        RetrievalChain: The context retriever chain.
    """
    llm = ChatOpenAI()
    retriever = vector_store.as_retriever()

    prompt = ChatPromptTemplate.from_messages([
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        ("user", "Given the above conversation, generate a search query "
                 "to look up in order to get information relevant to "
                 "the conversation")
    ])

    retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

    return retriever_chain


def get_conversational_rag_chain(retriever_chain):
    """Create a conversational RAG chain
    using the given context retriever chain.

    This function creates a conversational RAG chain
    using a context retriever chain and a chat prompt template.

    Args:
        retriever_chain (RetrievalChain): The context retriever chain.

    Returns:
        RetrievalChain: The conversational RAG chain.
    """
    llm = ChatOpenAI()

    prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            "Answer the user's questions based"
            " on the below context:\n\n{context}"
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}")
    ])

    stuff_documents_chain = create_stuff_documents_chain(llm, prompt)

    return create_retrieval_chain(retriever_chain, stuff_documents_chain)


def get_response(user_input):
    """Get the response for the given user query.

    This function retrieves the response for the user query
    using the conversational RAG chain.

    Args:
        user_input (str): The user query.

    Returns:
        str: The response to the user query.
    """
    # Create Conversation Chain
    retriever_chain = get_context_retriever_chain(
        st.session_state.vector_store
        )
    conversation_rag_chain = get_conversational_rag_chain(retriever_chain)

    # Invoke conversational RAG chain
    inv_response = conversation_rag_chain.invoke({
          "chat_history": st.session_state.chat_history,
          "input": user_input
      })
    return inv_response['answer']


# App Config
st.set_page_config(page_title="Chat  with Websites", page_icon="🦜")
st.title("_Chat_ with :blue[Websites] 🤖")

# App Sidebar
with st.sidebar:
    st.header(":red[Settings]", divider='rainbow')
    key = st.text_input("OpenAI API key 🔑", type="password")
    set_openai_key(key)
    website_url = st.text_input("Website URL 🌐")

if website_url is None or website_url == "":
    st.info("Please enter a website URL")
else:
    # For persistent variables : Session State
    if "chat_history" not in st.session_state:
        st.session_state.chat_history = [
            AIMessage(content="Yellow👋 I am a bot, Need help right ?")
            ]

    # Persistent Vector Store
    if "vector_store" not in st.session_state:
        st.session_state.vector_store = get_vectorstore_from_url(website_url)

    # User Input
    user_query = st.chat_input("Type your message ✍")
    if user_query is not None and user_query != "":
        response = get_response(user_query)
        st.session_state.chat_history.append(HumanMessage(content=user_query))
        st.session_state.chat_history.append(AIMessage(content=response))

    # Conversation
    for message in st.session_state.chat_history:
        MESSAGE_TYPE = "AI" if isinstance(message, AIMessage) else "Human"
        with st.chat_message(MESSAGE_TYPE):
            st.write(message.content)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


2024-12-21 16:52:09.669 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-21 16:52:09.671 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-21 16:52:09.675 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-21 16:52:09.677 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-21 16:52:09.679 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-21 16:52:09.680 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-21 16:52:09.682 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-21 16:52:09.683 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar